In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import string

In [ ]:
train=pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_train.csv",encoding='latin1')
test=pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_test.csv",encoding='latin1')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
data_train=train['OriginalTweet']
sentiment_train=train['Sentiment']
data_test=test['OriginalTweet']
sentiment_test=test['Sentiment']

In [ ]:
full_data=pd.concat([data_test,data_train])

# Sample Testing

In [ ]:
sample=data_train[:50]
sample_target=sentiment_train[:50]
sample_test=data_test[:10]

In [ ]:
from nltk.stem import WordNetLemmatizer
WNL=WordNetLemmatizer()

Removing punctuations, stopwords and lemmatizing

In [ ]:
def text_process(data):
    msg=[c for c in data if c not in string.punctuation]
    msg=''.join(msg)
    msg=[word for word in msg.split() if word.lower() not in stopwords.words('english')]
    msg=[WNL.lemmatize(word) for word in msg ]
    return msg

In [ ]:
sample

In [ ]:
sample.apply(text_process)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [ ]:
CV=CountVectorizer(analyzer=text_process)

In [ ]:
CV.fit(sample)

In [ ]:
CV.vocabulary_

In [ ]:
sample=CV.transform(sample)

In [ ]:
#non zero values
sample.nnz

In [ ]:
Tfidf=TfidfTransformer()
Tfidf.fit(sample)
Tfidf_val=Tfidf.transform(sample)

In [ ]:
sample_test=CV.transform(sample_test)
sample_test=Tfidf.transform(sample_test)

### a) Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
sentiment=MultinomialNB()
sentiment.fit(Tfidf_val,sample_target)
result1=sentiment.predict(sample_test)

In [ ]:
result1

### b) Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC=RandomForestClassifier()
RFC.fit(Tfidf_val,sample_target)
result2=RFC.predict(sample_test)

In [ ]:
result2

In [ ]:
sentiment_test.head(10)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
from sklearn.pipeline import Pipeline

# 1) With IDF

### a) Naive Bayes

In [ ]:
estimator1=[('CV',CountVectorizer(analyzer=text_process)),('Tfidf',TfidfTransformer()),('Final Analysis',MultinomialNB())]

In [ ]:
pipe1=Pipeline(estimator1)

In [ ]:
pipe1.fit(data_train,sentiment_train)

In [ ]:
predict1=pipe1.predict(data_test)

In [ ]:
print(classification_report(sentiment_test,predict1))

In [ ]:
print(confusion_matrix(sentiment_test,predict1))

In [ ]:
df1=pd.DataFrame()
df1['actual']=sentiment_test
df1['predicted']=predict1

In [ ]:
df1.head(15)

### b) Random Forest Classifier

In [ ]:
estimator2=[('CV',CountVectorizer(analyzer=text_process)),('Tfidf',TfidfTransformer()),
            ('Final Analysis',RandomForestClassifier())]

In [ ]:
pipe2=Pipeline(estimator2)

In [ ]:
pipe2.fit(data_train,sentiment_train)

In [ ]:
predict2=pipe2.predict(data_test)

In [ ]:
print(classification_report(sentiment_test,predict2))

In [ ]:
print(confusion_matrix(sentiment_test,predict2))

In [ ]:
df2=pd.DataFrame()
df2['actual']=sentiment_test
df2['predicted']=predict2

In [ ]:
df2.head(15)

# 2) Without IDF

### a) Naive Bayes

In [ ]:
estimator3=[('CV',CountVectorizer(analyzer=text_process)),('Tfidf',TfidfTransformer(use_idf=False)),
            ('Final Analysis',MultinomialNB())]

In [ ]:
pipe3=Pipeline(estimator3)

In [ ]:
pipe3.fit(data_train,sentiment_train)

In [ ]:
predict3=pipe3.predict(data_test)

In [ ]:
print(classification_report(sentiment_test,predict3))

In [ ]:
print(confusion_matrix(sentiment_test,predict3))

In [ ]:
df3=pd.DataFrame()
df3['actual']=sentiment_test
df3['predicted']=predict3

In [ ]:
df3.head(15)

### b) Random Forest Classifier

In [ ]:
estimator4=[('CV',CountVectorizer(analyzer=text_process)),('Tfidf',TfidfTransformer(use_idf=False)),
            ('Final Analysis',RandomForestClassifier())]

In [ ]:
pipe4=Pipeline(estimator4)

In [ ]:
pipe4.fit(data_train,sentiment_train)

In [ ]:
predict4=pipe4.predict(data_test)

In [ ]:
print(classification_report(sentiment_test,predict4))

In [ ]:
print(confusion_matrix(sentiment_test,predict4))

In [ ]:
df4=pd.DataFrame()
df4['actual']=sentiment_test
df4['predicted']=predict4

In [ ]:
df4.head(15)

# Conclusion

### Best Prediction By:  Random Forest Classifier

# Word Cloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
def one_sentence(data):
    msg=' '.join(data)
    return msg
def join_all(data):
    msg = " "
    for i in data:
        msg = msg + i
    return msg

In [ ]:
full_data=full_data.apply(text_process)
full_data=full_data.apply(one_sentence)

In [ ]:
whole=join_all(full_data)

In [ ]:
whole

In [ ]:
plt.figure(figsize=(18,10))
wordcloud=WordCloud(background_color='white').generate(whole)
plt.imshow(wordcloud)
plt.axis('off')